https://medium.com/psicodata/baixando-e-processando-dados-do-datasus-sobre-suic%C3%ADdio-com-python-656afa17f6ad

In [1]:
from pysus.online_data.SIM import download
import pandas as pd
import datetime
estados = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT','MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC','SP', 'SE', 'TO']
anos = [2019, 2020]

In [2]:
banco = {}
for y in anos:
    for uf in estados:
        banco[uf, y] = download(state=uf, year=y)
        print("Banco de " + str(y) + " de " + str(uf) + " baixado!")

Banco de 2019 de AC baixado!
Banco de 2019 de AL baixado!
Banco de 2019 de AP baixado!
Banco de 2019 de AM baixado!
Banco de 2019 de BA baixado!
Banco de 2019 de CE baixado!
Banco de 2019 de DF baixado!
Banco de 2019 de ES baixado!
Banco de 2019 de GO baixado!
Banco de 2019 de MA baixado!
Banco de 2019 de MT baixado!
Banco de 2019 de MS baixado!
Banco de 2019 de MG baixado!
Banco de 2019 de PA baixado!
Banco de 2019 de PB baixado!
Banco de 2019 de PR baixado!
Banco de 2019 de PE baixado!
Banco de 2019 de PI baixado!
Banco de 2019 de RJ baixado!
Banco de 2019 de RN baixado!
Banco de 2019 de RS baixado!
Banco de 2019 de RO baixado!
Banco de 2019 de RR baixado!
Banco de 2019 de SC baixado!
Banco de 2019 de SP baixado!
Banco de 2019 de SE baixado!
Banco de 2019 de TO baixado!
Banco de 2020 de AC baixado!
Banco de 2020 de AL baixado!
Banco de 2020 de AP baixado!
Banco de 2020 de AM baixado!
Banco de 2020 de BA baixado!
Banco de 2020 de CE baixado!
Banco de 2020 de DF baixado!
Banco de 2020 

In [3]:
todos = pd.concat({k: pd.DataFrame.from_dict(v) for k, v in banco.items()}, axis=0).reset_index()
todos = todos[[
'level_0', 'level_1', 'CIRCOBITO', 'DTOBITO', 'DTNASC', 'SEXO', 'RACACOR', 'ESTCIV', 'ESC', 'OCUP', 'CODMUNRES', 'LOCOCOR', 'ASSISTMED', 'CAUSABAS', 'CAUSABAS_O'
]]
todos.rename(columns={'level_0': 'estado', 'level_1': 'ano'}, inplace=True)  # renomeando colunas


In [4]:
filter_list = ["X{}".format(x) for x in range(600, 850)]
cid10 = todos[
todos['CAUSABAS'].isin(filter_list) | todos['CAUSABAS_O'].isin(filter_list)
]

In [5]:
dictCIRCOBITO = {
'1': 'Acidente',
'2': 'Suicídio',
'3': 'Homicídio',
'4': 'Outro',
'0': 'NA',
'6': "NA",
'7': "NA",
'8': "NA",
'9': "NA"
}
dicSEXO = {
'1': 'Masculino',
'2': 'Feminino',
'0': 'NA',
'9': 'NA'
}
dicRACACOR = {
'1': 'Branca',
'2': 'Preta',
'3': 'Amarela',
'4': 'Parda',
'5': 'Indígena',
'0': 'NA',
'6': "NA",
'7': "NA",
'8': "NA",
'9': "NA"
}
dicESTCIV = {
'1': 'Solteiro',
'2': 'Casado',
'3': 'Viúvo',
'4': 'Separado judicialmente',
'5': 'União consensual',
'0': 'NA',
'6': "NA",
'7': "NA",
'8': "NA",
'9': "NA"
}
dicESC = {
'1': 'Nenhuma',
'2': '1 a 3 anos',
'3': '4 a 7 anos',
'4': '8 a 11 anos',
'5': '12 e mais',
'8': 'De 9 a 11 anos',
'0': 'NA',
'6': 'NA',
'7': 'NA',
'9': 'NA',
'': 'NA'
}
dictLOCOCOR = {
'1': 'Hospital',
'2': 'Outro estabelecimento de saúde',
'3': 'Domicílio',
'4': 'Via pública',
'5': 'Outros',
'9': 'NA'
}
dictASSISTMED = {
'1': 'Sim',
'2': 'Não',
'9': 'NA'
}
cid10 = cid10.replace({'CIRCOBITO': dictCIRCOBITO, 'SEXO': dicSEXO, 'RACACOR': dicRACACOR,'ESTCIV': dicESTCIV, 'ESC': dicESC, 'LOCOCOR': dictLOCOCOR, 'ASSISTMED': dictASSISTMED})
cid10['DTNASC'] = cid10['DTNASC'][cid10['DTNASC'] != ''].apply(lambda x: datetime.datetime.strptime(x, "%d%m%Y"))
cid10['DTOBITO'] = pd.to_datetime(cid10['DTOBITO'], format="%d%m%Y")

In [6]:
cid10['idade'] = cid10['DTOBITO'].dt.year - cid10['DTNASC'].dt.year
cid10['mes'] = cid10['DTOBITO'].map(lambda x: x.strftime('%m'))

In [7]:
CBO2002 = pd.read_csv('CBO.csv').set_index('CODIGO').to_dict()
cid10['OCUP'] = cid10['OCUP'].replace('', 0).astype(int)
cid10['OCUP'] = cid10['OCUP'].replace(CBO2002['OCUPACAO'])

In [8]:
URL = 'http://blog.mds.gov.br/redesuas/wp-content/uploads/2018/06/Lista_Munic%C3%ADpios_com_IBGE_Brasil_Versao_CSV.csv'
municipios = pd.read_csv(URL, error_bad_lines=False, sep=';', encoding='latin-1', usecols=['IBGE', 'IBGE7']).set_index('IBGE').to_dict()
cid10['CODMUNRES'] = cid10['CODMUNRES'].astype(int).replace(municipios['IBGE7'])

/tmp/ipykernel_3462071/366119948.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  municipios = pd.read_csv(URL, error_bad_lines=False, sep=';', encoding='latin-1', usecols=['IBGE', 'IBGE7']).set_index('IBGE').to_dict()


In [9]:
todos.drop(['ano', 'estado', 'CIRCOBITO', 'DTNASC', 'SEXO', 'RACACOR', 'ESTCIV', 'ESC', 'OCUP', 'LOCOCOR', 'ASSISTMED', 'CAUSABAS', 'CAUSABAS_O'], axis=1, inplace=True)

In [10]:
todos['DTOBITO'] = pd.to_datetime(todos['DTOBITO'], format="%d%m%Y")

In [11]:
cid10.to_csv('/home/diego/Desktop/monografia/raw_data/suicidios/suicidios.csv', index=False)
todos.to_csv('/home/diego/Desktop/monografia/raw_data/suicidios/todos.csv', index=False)

### Casos de Doenças Mentais

In [1]:
import requests
import bs4
import pandas as pd
from io import StringIO

%cd ..

/home/diego/Desktop/monografia


In [4]:
df_all = pd.DataFrame(columns=['Município', 'Menor 1 ano', '1 a 4 anos', '5 a 9 anos', '10 a 14 anos',
       '15 a 19 anos', '20 a 24 anos', '25 a 29 anos', '30 a 34 anos',
       '35 a 39 anos', '40 a 44 anos', '45 a 49 anos', '50 a 54 anos',
       '55 a 59 anos', '60 a 64 anos', '65 a 69 anos', '70 a 74 anos',
       '75 a 79 anos', '80 anos e mais', 'Total', 'municipio_ibge6', 'nome',
       'date'])

for ano in ['19', '20']:
    for mes in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
        cookies = {
            'TS014879da': '01e046ca4c8769ecdb02c194221097e0e61da7de5335bbb2cb64d8a3bdb9d428eb5b8d2caa21ebfd8cc3e394f419557e59a7a6b231',
        }

        headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
            'Cache-Control': 'max-age=0',
            'Connection': 'keep-alive',
            'Content-Type': 'application/x-www-form-urlencoded',
            # 'Cookie': 'TS014879da=01e046ca4c8769ecdb02c194221097e0e61da7de5335bbb2cb64d8a3bdb9d428eb5b8d2caa21ebfd8cc3e394f419557e59a7a6b231',
            'Origin': 'http://tabnet.datasus.gov.br',
            'Referer': 'http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sia/cnv/qbbr.def',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
            'dnt': '1',
            'sec-gpc': '1',
        }

        data = f'Linha=Munic%EDpio&Coluna=Faixa_et%E1ria&Incremento=Qtd.aprovada&Arquivos=qbbr{ano}{mes}.dbf&pesqmes1=Digite+o+texto+e+ache+f%E1cil&SMunic%EDpio=TODAS_AS_CATEGORIAS__&pesqmes2=Digite+o+texto+e+ache+f%E1cil&SCapital=TODAS_AS_CATEGORIAS__&pesqmes3=Digite+o+texto+e+ache+f%E1cil&SRegi%E3o_de_Sa%FAde_%28CIR%29=TODAS_AS_CATEGORIAS__&pesqmes4=Digite+o+texto+e+ache+f%E1cil&SMacrorregi%E3o_de_Sa%FAde=TODAS_AS_CATEGORIAS__&pesqmes5=Digite+o+texto+e+ache+f%E1cil&SMicrorregi%E3o_IBGE=TODAS_AS_CATEGORIAS__&pesqmes6=Digite+o+texto+e+ache+f%E1cil&SRegi%E3o_Metropolitana_-_RIDE=TODAS_AS_CATEGORIAS__&pesqmes7=Digite+o+texto+e+ache+f%E1cil&STerrit%F3rio_da_Cidadania=TODAS_AS_CATEGORIAS__&pesqmes8=Digite+o+texto+e+ache+f%E1cil&SMesorregi%E3o_PNDR=TODAS_AS_CATEGORIAS__&SAmaz%F4nia_Legal=TODAS_AS_CATEGORIAS__&SSemi%E1rido=TODAS_AS_CATEGORIAS__&SFaixa_de_Fronteira=TODAS_AS_CATEGORIAS__&SZona_de_Fronteira=TODAS_AS_CATEGORIAS__&SMunic%EDpio_de_extrema_pobreza=TODAS_AS_CATEGORIAS__&pesqmes14=Digite+o+texto+e+ache+f%E1cil&SProcedimento=TODAS_AS_CATEGORIAS__&SGrupo_procedimento=TODAS_AS_CATEGORIAS__&pesqmes16=Digite+o+texto+e+ache+f%E1cil&SSubgrupo_proced.=TODAS_AS_CATEGORIAS__&pesqmes17=Digite+o+texto+e+ache+f%E1cil&SForma_organiza%E7%E3o=TODAS_AS_CATEGORIAS__&SComplexidade=TODAS_AS_CATEGORIAS__&SCar%E1ter_Atendiment=TODAS_AS_CATEGORIAS__&SDocumento_registro=7&pesqmes21=Digite+o+texto+e+ache+f%E1cil&SFaixa_et%E1ria=TODAS_AS_CATEGORIAS__&SSexo=TODAS_AS_CATEGORIAS__&pesqmes23=Digite+o+texto+e+ache+f%E1cil&SProfissional_-_CBO=TODAS_AS_CATEGORIAS__&zeradas=exibirlz&formato=prn&mostre=Mostra'

        response_psico = requests.post('http://tabnet.datasus.gov.br/cgi/tabcgi.exe?sia/cnv/qbbr.def', cookies=cookies, headers=headers, data=data, verify=False)

        cookies = {
            'TS014879da': '01e046ca4c35857864bae7a51a6f49155d0802970c8881d7555b468f0c1ab498669269934a032389998f439b887b481a8177af6e49',
        }

        headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
            'Cache-Control': 'max-age=0',
            'Connection': 'keep-alive',
            'Content-Type': 'application/x-www-form-urlencoded',
            # 'Cookie': 'TS014879da=01e046ca4c35857864bae7a51a6f49155d0802970c8881d7555b468f0c1ab498669269934a032389998f439b887b481a8177af6e49',
            'Origin': 'http://tabnet.datasus.gov.br',
            'Referer': 'http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sia/cnv/qbbr.def',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
            'dnt': '1',
            'sec-gpc': '1',
        }

        data = f'Linha=Munic%EDpio&Coluna=--N%E3o-Ativa--&Incremento=Qtd.aprovada&Arquivos=qbbr{ano}{mes}.dbf&pesqmes1=Digite+o+texto+e+ache+f%E1cil&SMunic%EDpio=TODAS_AS_CATEGORIAS__&pesqmes2=Digite+o+texto+e+ache+f%E1cil&SCapital=TODAS_AS_CATEGORIAS__&pesqmes3=Digite+o+texto+e+ache+f%E1cil&SRegi%E3o_de_Sa%FAde_%28CIR%29=TODAS_AS_CATEGORIAS__&pesqmes4=Digite+o+texto+e+ache+f%E1cil&SMacrorregi%E3o_de_Sa%FAde=TODAS_AS_CATEGORIAS__&pesqmes5=Digite+o+texto+e+ache+f%E1cil&SMicrorregi%E3o_IBGE=TODAS_AS_CATEGORIAS__&pesqmes6=Digite+o+texto+e+ache+f%E1cil&SRegi%E3o_Metropolitana_-_RIDE=TODAS_AS_CATEGORIAS__&pesqmes7=Digite+o+texto+e+ache+f%E1cil&STerrit%F3rio_da_Cidadania=TODAS_AS_CATEGORIAS__&pesqmes8=Digite+o+texto+e+ache+f%E1cil&SMesorregi%E3o_PNDR=TODAS_AS_CATEGORIAS__&SAmaz%F4nia_Legal=TODAS_AS_CATEGORIAS__&SSemi%E1rido=TODAS_AS_CATEGORIAS__&SFaixa_de_Fronteira=TODAS_AS_CATEGORIAS__&SZona_de_Fronteira=TODAS_AS_CATEGORIAS__&SMunic%EDpio_de_extrema_pobreza=TODAS_AS_CATEGORIAS__&pesqmes14=Digite+o+texto+e+ache+f%E1cil&SProcedimento=TODAS_AS_CATEGORIAS__&SGrupo_procedimento=TODAS_AS_CATEGORIAS__&pesqmes16=Digite+o+texto+e+ache+f%E1cil&SSubgrupo_proced.=TODAS_AS_CATEGORIAS__&pesqmes17=Digite+o+texto+e+ache+f%E1cil&SForma_organiza%E7%E3o=TODAS_AS_CATEGORIAS__&SComplexidade=TODAS_AS_CATEGORIAS__&SCar%E1ter_Atendiment=TODAS_AS_CATEGORIAS__&SDocumento_registro=TODAS_AS_CATEGORIAS__&pesqmes21=Digite+o+texto+e+ache+f%E1cil&SFaixa_et%E1ria=TODAS_AS_CATEGORIAS__&SSexo=TODAS_AS_CATEGORIAS__&pesqmes23=Digite+o+texto+e+ache+f%E1cil&SProfissional_-_CBO=TODAS_AS_CATEGORIAS__&zeradas=exibirlz&formato=prn&mostre=Mostra'

        response_total = requests.post('http://tabnet.datasus.gov.br/cgi/tabcgi.exe?sia/cnv/qbbr.def', cookies=cookies, headers=headers, data=data)
        
        document = bs4.BeautifulSoup(response_psico.text, 'html.parser')
        data = document.find('pre')
        TESTDATA = StringIO(data.get_text())
        df_psico = pd.read_csv(TESTDATA, sep=";")
        
        document = bs4.BeautifulSoup(response_total.text, 'html.parser')
        data = document.find('pre')
        TESTDATA = StringIO(data.get_text())
        df_total = pd.read_csv(TESTDATA, sep=";")

        df_psico = df_psico[(df_psico['Município'] != 'Total') & (df_psico['Município'] != '&')]
        df_total = df_total[(df_total['Município'] != 'Total') & (df_total['Município'] != '&')]
        
        df_psico['municipio_ibge6'] = df_psico['Município'].apply(lambda x: x.split(' ')[0])
        df_psico['nome'] = df_psico['Município'].apply(lambda x: ' '.join(x.split(' ')[1:]))
        df_total['municipio_ibge6'] = df_total['Município'].apply(lambda x: x.split(' ')[0])
        df_total['nome'] = df_psico['Município'].apply(lambda x: ' '.join(x.split(' ')[1:]))

        df_psico = df_psico.replace('-', 0)
        df_total = df_total.replace('-', 0)

        df = pd.merge(df_psico, df_total[['municipio_ibge6', 'Qtd.aprovada']], on='municipio_ibge6')

        df = df[['Município', 'Menor 1 ano', '1 a 4 anos', '5 a 9 anos', '10 a 14 anos',
       '15 a 19 anos', '20 a 24 anos', '25 a 29 anos', '30 a 34 anos',
       '35 a 39 anos', '40 a 44 anos', '45 a 49 anos', '50 a 54 anos',
       '55 a 59 anos', '60 a 64 anos', '65 a 69 anos', '70 a 74 anos',
       '75 a 79 anos', '80 anos e mais', 'Total', 'municipio_ibge6', 'nome',
       'Qtd.aprovada']]

        df.rename(columns={'Qtd.aprovada': 'total_internacoes_geral'}, inplace=True)

        df['date'] = f'20{ano}-{mes}-01'

        df_all = pd.concat([df_all, df], ignore_index=True)

        df_all.to_csv('raw_data/internacoes/internacoes.csv', index=False)